<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [4]:
# Create a base map centered around a specific location
m = folium.Map(location=[34.5, -118.5], zoom_start=6)  # Adjust the center as needed

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(m)

# List of launch sites with their names and coordinates
launch_sites = [
    {'name': 'Cape Canaveral Space Force Station', 'location': [28.3922, -80.6077]},
    {'name': 'Kennedy Space Center', 'location': [28.5721, -80.6480]},
    {'name': 'Vandenberg Space Force Base', 'location': [34.7333, -120.5735]},
    {'name': 'Space Launch Complex 40', 'location': [28.5623, -80.5774]},
    {'name': 'Space Launch Complex 41', 'location': [28.5623, -80.5774]},
    # Add more launch sites as needed
]

# Add markers for each launch site
for site in launch_sites:
    folium.Marker(
        location=site['location'],
        popup=site['name'],
        icon=folium.Icon(color='blue')
    ).add_to(marker_cluster)

# Save the map to an HTML file
m.save("launch_sites_map.html")

# Display the map (if applicable in Jupyter notebook)
m

First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
# Initial the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
# Loop through each launch site and add a circle and marker
for index, row in launch_sites_df.iterrows():
    # Create a circle for each launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,  # radius in meters
        color='#000000',  # black circle
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

    # Create a marker for each launch site
    folium.map.Marker(
        location=[row['Lat'], row['Long']],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#000000;"><b>%s</b></div>' % row['Launch Site'],
        )
    ).add_to(site_map)

# Save the map to an HTML file
site_map.save("spacex_launch_sites_map_with_circles.html")

# Display the map (if applicable in Jupyter notebook)
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [10]:
# Task 2: Mark the success/failed launches for each site on the map
# Assuming you already have spacex_df with launch outcomes
# Create a base map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Add a circle and marker for NASA Johnson Space Center
folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center')).add_to(site_map)
folium.map.Marker(
    nasa_coordinate,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
).add_to(site_map)

# Loop through each launch site and add circles and markers
for index, row in launch_sites_df.iterrows():
    # Create a circle for each launch site
    folium.Circle(
        location=[row['Lat'], row['Long']],
        radius=1000,
        color='#000000',  # black circle
        fill=True
    ).add_child(folium.Popup(row['Launch Site'])).add_to(site_map)

    # Filter launch outcomes for the specific launch site
    site_launches = spacex_df[spacex_df['Launch Site'] == row['Launch Site']]

    # Add markers for each launch
    for _, launch in site_launches.iterrows():
        color = 'green' if launch['class'] == 1 else 'red'  # Assuming 'class' is 1 for success and 0 for failure
        folium.Marker(
            location=[launch['Lat'], launch['Long']],
            icon=folium.Icon(color=color)
        ).add_to(site_map)

# Save the updated map
site_map.save("spacex_launch_sites_with_outcomes_map.html")

# Display the map (if applicable in Jupyter notebook)
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [12]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [13]:
# Create a base map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# TODO 1: Create a new column in spacex_df called marker_color
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# TODO 2: Add folium.Marker to marker_cluster for each launch result
for _, launch in spacex_df.iterrows():
    folium.Marker(
        location=[launch['Lat'], launch['Long']],
        icon=folium.Icon(color=launch['marker_color'])
    ).add_to(marker_cluster)

# Save the updated map
site_map.save("spacex_launch_records_map.html")

# Display the map (if applicable in Jupyter notebook)
site_map


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [14]:
# Assuming previous imports and spacex_df setup are already done

# Create a base map centered at NASA Johnson Space Center
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Create a MarkerCluster object
marker_cluster = MarkerCluster().add_to(site_map)

# TODO 1: Create a new column in spacex_df called marker_color
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

# TODO 2: Add marker_cluster to current site_map and create markers for each launch
for index, record in spacex_df.iterrows():
    # Create a Marker object with its coordinates
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color'])  # Customize icon color based on outcome
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Display the updated map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [15]:
import numpy as np

# Define a function to calculate the Haversine distance
def haversine(lat1, lon1, lat2, lon2):
    r = 6371  # Radius of the Earth in kilometers
    # Convert latitude and longitude from degrees to radians
    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    # Haversine formula
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    a = (np.sin(dlat / 2) ** 2 +
         np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2) ** 2)
    c = 2 * np.arcsin(np.sqrt(a))
    distance = r * c  # Distance in kilometers
    return distance

# Example distances calculation to a set of proximities
proximity_points = {
    "Cape Canaveral": [28.3922, -80.6077],
    "Kennedy Space Center": [28.5721, -80.6480],
    "Vandenberg": [34.7333, -120.5735],
    "SpaceX Launch Complex 40": [28.5623, -80.5774]
}

# Calculate distances from each launch site
launch_distances = {}

for index, row in launch_sites_df.iterrows():
    launch_site_name = row['Launch Site']
    launch_lat = row['Lat']
    launch_lon = row['Long']
    distances = {}
    
    for site_name, coords in proximity_points.items():
        distance = haversine(launch_lat, launch_lon, coords[0], coords[1])
        distances[site_name] = distance
        
    launch_distances[launch_site_name] = distances

# Display the calculated distances
launch_distances


{'CCAFS LC-40': {'Cape Canaveral': 19.145584595915437,
  'Kennedy Space Center': 6.984382038064585,
  'Vandenberg': 3822.2298375684654,
  'SpaceX Launch Complex 40': 0.004255902457358078},
 'CCAFS SLC-40': {'Cape Canaveral': 19.25208049837064,
  'Kennedy Space Center': 7.021379973825824,
  'Vandenberg': 3822.243696288376,
  'SpaceX Launch Complex 40': 0.11471976840424548},
 'KSC LC-39A': {'Cape Canaveral': 20.493570592440303,
  'Kennedy Space Center': 0.1676891412798242,
  'Vandenberg': 3815.446206858494,
  'SpaceX Launch Complex 40': 6.895159766430582},
 'VAFB SLC-4E': {'Cape Canaveral': 3829.725900417789,
  'Kennedy Space Center': 3818.9966782771126,
  'Vandenberg': 11.678868270153693,
  'SpaceX Launch Complex 40': 3825.836374583027}}

Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [16]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [17]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [18]:
import folium
from folium.plugins import MousePosition, MarkerCluster

# Create a base map centered at NASA Johnson Space Center (example coordinates)
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=6)

# Add MousePosition to the map to get lat/lon data dynamically
MousePosition().add_to(site_map)

# Create a MarkerCluster object for clustering multiple markers
marker_cluster = MarkerCluster().add_to(site_map)

# Define the closest coastline coordinates (corrected)
coastline_lat = 28.56338
coastline_lon = -80.56792

# Create a Marker for the coastline point
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline Point',
    icon=folium.Icon(color='blue')
).add_to(site_map)

# Adjust column name if necessary for success/failure (assuming it's called 'class')
success_column = 'class'  # Adjust this based on your actual column name

# Loop through each record in spacex_df data frame
for index, record in spacex_df.iterrows():
    # Calculate distance to the coastline
    distance_coastline = calculate_distance(record['Lat'], record['Long'], coastline_lat, coastline_lon)
    
    # Create a Marker object for each launch site
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        popup=f'{record["Launch Site"]}: {distance_coastline:.2f} km to coastline',
        icon=folium.Icon(color='green' if record[success_column] == 1 else 'red')
    )
    
    # Add the marker to the marker cluster
    marker_cluster.add_child(marker)

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Save the updated map
site_map.save("spacex_launch_sites_with_coastline.html")

# Display the map (if applicable in Jupyter notebook)
site_map


In [19]:
launch_sites_df['Launch Site']

0     CCAFS LC-40
1    CCAFS SLC-40
2      KSC LC-39A
3     VAFB SLC-4E
Name: Launch Site, dtype: object

In [20]:
# Correct coastline coordinates
coastline_lat = 28.56338
coastline_lon = -80.56792

# Define the specific launch site for distance calculation
launch_site_name = "CCAFS LC-40"  # Name from your dataset
matching_sites = launch_sites_df[launch_sites_df['Launch Site'].str.strip() == launch_site_name]

if not matching_sites.empty:
    launch_site_record = matching_sites.iloc[0]
    
    # Calculate distance from the launch site to the coastline
    distance = calculate_distance(launch_site_record['Lat'], launch_site_record['Long'], coastline_lat, coastline_lon)

    # Create and add a folium.Marker on the closest coastline point
    distance_marker = folium.Marker(
        location=[coastline_lat, coastline_lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12px; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance),
        )
    )
    
    # Add the distance marker to the site map
    site_map.add_child(distance_marker)

    # Add a PolyLine between the launch site and the coastline for better visualization
    folium.PolyLine(
        locations=[[launch_site_record['Lat'], launch_site_record['Long']], [coastline_lat, coastline_lon]],
        color='blue',
        weight=2.5,
        opacity=0.8
    ).add_to(site_map)

    # Optionally, you can also add markers and lines for the railway and highway proximity:
    
    # Closest railway coordinates
    railway_lat = 28.56376
    railway_lon = -80.58685
    folium.Marker(
        location=[railway_lat, railway_lon],
        popup='Closest Railway',
        icon=folium.Icon(color='purple')
    ).add_to(site_map)
    
    # Add PolyLine for railway
    folium.PolyLine(
        locations=[[launch_site_record['Lat'], launch_site_record['Long']], [railway_lat, railway_lon]],
        color='purple',
        weight=2.5,
        opacity=0.8
    ).add_to(site_map)

    # Closest highway coordinates
    highway_lat = 28.51327
    highway_lon = -80.61115
    folium.Marker(
        location=[highway_lat, highway_lon],
        popup='Closest Highway',
        icon=folium.Icon(color='orange')
    ).add_to(site_map)

    # Add PolyLine for highway
    folium.PolyLine(
        locations=[[launch_site_record['Lat'], launch_site_record['Long']], [highway_lat, highway_lon]],
        color='orange',
        weight=2.5,
        opacity=0.8
    ).add_to(site_map)

else:
    print(f"No matching launch site found for: {launch_site_name}")

# Save the updated map
site_map.save("spacex_launch_sites_with_proximities.html")

# Display the map (if applicable in Jupyter notebook)
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [21]:
# Correct coastline coordinates
coastline_lat = 28.56338  # Correct latitude of the coastline
coastline_lon = -80.56792  # Correct longitude of the coastline

# Define the launch site name for which you want to draw the PolyLine
launch_site_name = "CCAFS LC-40"  # Launch site name from the DataFrame

# Find the matching site in the DataFrame
matching_sites = launch_sites_df[launch_sites_df['Launch Site'].str.strip() == launch_site_name]

if not matching_sites.empty:
    # Get the first matching record
    launch_site_record = matching_sites.iloc[0]
    
    # Extract the coordinates of the launch site
    launch_lat = launch_site_record['Lat']
    launch_lon = launch_site_record['Long']
    
    # Create a list of coordinates to define the PolyLine
    coordinates = [(launch_lat, launch_lon), (coastline_lat, coastline_lon)]
    
    # Create a PolyLine between the launch site and the coastline
    lines = folium.PolyLine(locations=coordinates, weight=2, color='blue', opacity=0.7)
    
    # Add the PolyLine to the map
    site_map.add_child(lines)
    
    # Add a marker for the launch site
    folium.Marker(
        location=[launch_lat, launch_lon],
        popup=launch_site_name,  # Popup with launch site name
        icon=folium.Icon(color='green')  # Green marker for the launch site
    ).add_to(site_map)
    
    # Add a marker for the closest coastline point
    folium.Marker(
        location=[coastline_lat, coastline_lon],
        popup='Closest Coastline Point',  # Popup text for coastline
        icon=folium.Icon(color='blue')  # Blue marker for the coastline
    ).add_to(site_map)

else:
    # If no matching site is found, print a message
    print(f"No matching launch site found for: {launch_site_name}")

# Save the updated map to an HTML file
site_map.save("spacex_launch_sites_with_polyline.html")

# Display the map (if applicable in a Jupyter notebook)
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [22]:
# Assuming you have already created the site_map and added the MousePosition

# Updated coordinates based on proximity data
closest_city_coords = (28.3922, -80.6077)  # Cape Canaveral City coordinates
closest_railway_coords = (28.56376, -80.58685)  # Closest railway coordinates
closest_highway_coords = (28.51327, -80.61115)  # Closest highway coordinates

# Define the launch site name for which you want to draw the PolyLine
launch_site_name = "CCAFS LC-40"  # Adjust based on actual launch site name
matching_sites = launch_sites_df[launch_sites_df['Launch Site'].str.strip() == launch_site_name]

if not matching_sites.empty:
    launch_site_record = matching_sites.iloc[0]
    
    # Coordinates of the launch site
    launch_lat = launch_site_record['Lat']
    launch_lon = launch_site_record['Long']
    
    # Draw PolyLine to the closest city
    city_line = folium.PolyLine(
        locations=[(launch_lat, launch_lon), closest_city_coords],
        color='red',  # Color for city line
        weight=2,
        opacity=0.7
    )
    site_map.add_child(city_line)

    # Draw PolyLine to the closest railway
    railway_line = folium.PolyLine(
        locations=[(launch_lat, launch_lon), closest_railway_coords],
        color='purple',  # Color for railway line
        weight=2,
        opacity=0.7
    )
    site_map.add_child(railway_line)

    # Draw PolyLine to the closest highway
    highway_line = folium.PolyLine(
        locations=[(launch_lat, launch_lon), closest_highway_coords],
        color='orange',  # Color for highway line
        weight=2,
        opacity=0.7
    )
    site_map.add_child(highway_line)

# Save the updated map
site_map.save("spacex_launch_sites_with_lines_to_proximities.html")

# Display the map (if applicable in Jupyter notebook)
site_map


In [23]:
# Coordinates for CCAFS LC-40
launch_site_lat = 28.5623  # Latitude for CCAFS LC-40
launch_site_lon = -80.5774  # Longitude for CCAFS LC-40

# Updated coordinates for points of interest
closest_railway_coords = [28.56376, -80.58685]  # Closest railway
closest_highway_coords = [28.51327, -80.61115]  # Closest highway
closest_coastline_coords = [28.56338, -80.56792]  # Closest coastline
closest_city_coords = [28.3922, -80.6077]  # Cape Canaveral City

# Calculate distances to points of interest
distance_to_railway = calculate_distance(launch_site_lat, launch_site_lon, *closest_railway_coords)
distance_to_highway = calculate_distance(launch_site_lat, launch_site_lon, *closest_highway_coords)
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, *closest_coastline_coords)
distance_to_city = calculate_distance(launch_site_lat, launch_site_lon, *closest_city_coords)

# Proximity thresholds
proximity_threshold = 2.0  # km for railways and highways
coastline_threshold = 1.0   # km for coastline
city_threshold = 5.0        # km for cities

# Analysis for CCAFS LC-40
print("Analysis for CCAFS LC-40:")

# Check proximity to railways
if distance_to_railway <= proximity_threshold:
    print(f" - Close proximity to railways: {distance_to_railway:.2f} km")
else:
    print(f" - Not close to railways: {distance_to_railway:.2f} km")

# Check proximity to highways
if distance_to_highway <= proximity_threshold:
    print(f" - Close proximity to highways: {distance_to_highway:.2f} km")
else:
    print(f" - Not close to highways: {distance_to_highway:.2f} km")

# Check proximity to coastline
if distance_to_coastline <= coastline_threshold:
    print(f" - Close proximity to coastline: {distance_to_coastline:.2f} km")
else:
    print(f" - Not close to coastline: {distance_to_coastline:.2f} km")

# Check distance from cities
if distance_to_city >= city_threshold:
    print(f" - Kept distance away from cities: {distance_to_city:.2f} km")
else:
    print(f" - Close to cities: {distance_to_city:.2f} km")


Analysis for CCAFS LC-40:
 - Close proximity to railways: 0.94 km
 - Not close to highways: 6.37 km
 - Close proximity to coastline: 0.93 km
 - Kept distance away from cities: 19.15 km


In [24]:
# Example distances for CCAFS LC-40 (replace with actual calculated distances)
railway_distance = 0.94  # km to the nearest railway
highway_distance = 6.37  # km to the nearest highway
coastline_distance = 0.93  # km to the nearest coastline
city_distance = 19.15  # km to the nearest city

# Proximity thresholds
proximity_threshold = 2.0  # km for railways and highways
coastline_threshold = 1.0  # km for coastline
city_threshold = 5.0  # km for cities

# Analysis for CCAFS LC-40
launch_site = "CCAFS LC-40"
print(f"Analysis for {launch_site}:")

# Check proximity to railways
if railway_distance <= proximity_threshold:
    print(f" - Close proximity to railways: {railway_distance:.2f} km")
else:
    print(f" - Not close to railways: {railway_distance:.2f} km")

# Check proximity to highways
if highway_distance <= proximity_threshold:
    print(f" - Close proximity to highways: {highway_distance:.2f} km")
else:
    print(f" - Not close to highways: {highway_distance:.2f} km")

# Check proximity to coastline
if coastline_distance <= coastline_threshold:
    print(f" - Close proximity to coastline: {coastline_distance:.2f} km")
else:
    print(f" - Not close to coastline: {coastline_distance:.2f} km")

# Check distance from cities
if city_distance >= city_threshold:
    print(f" - Kept distance away from cities: {city_distance:.2f} km")
else:
    print(f" - Close to cities: {city_distance:.2f} km")


Analysis for CCAFS LC-40:
 - Close proximity to railways: 0.94 km
 - Not close to highways: 6.37 km
 - Close proximity to coastline: 0.93 km
 - Kept distance away from cities: 19.15 km


After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


Are launch sites in close proximity to railways?

Yes: The CCAFS LC-40 site is in close proximity to the nearest railway, with a distance of 0.94 km. This proximity allows for easy transportation of equipment and personnel, aiding logistics and operations at the launch site.
Are launch sites in close proximity to highways?

No: The nearest highway is 6.37 km away from CCAFS LC-40, which is beyond the proximity threshold of 2 km. Although not very close, the highway is still accessible and may require additional logistical planning for transporting heavy equipment.
Are launch sites in close proximity to coastline?

Yes: The CCAFS LC-40 site is very close to the coastline, with a distance of 0.93 km. This is important for safety, as launches typically take place near coastlines to minimize risk to populated areas and ensure safer launch and landing operations.
Do launch sites keep a certain distance away from cities?

Yes: The distance to the nearest city (Cape Canaveral) is 19.15 km, ensuring that the site adheres to safety regulations that require launch facilities to be located far from urban populations to reduce risks in case of launch accidents.

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
